In [1]:
from datasets import load_dataset
docs = load_dataset(f"mhhmm/leetcode-solutions-python", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to C:/Users/Tommaso/.cache/huggingface/datasets/mhhmm___json/mhhmm--leetcode-solutions-python-c6d2758e3a6cc905/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


In [2]:
docs.column_names

['id', 'code_only', 'code_with_problem', 'code_with_data', 'explanation_only']

In [3]:
docs.shape

(2359, 5)

In [9]:
docs["code_only"][1]

'```python\nclass ListNode:\n    def __init__(self, val=0, next=None):\n        self.val = val\n        self.next = next\n\ndef addTwoNumbers(l1, l2):\n    dummy = ListNode(0)\n    current = dummy\n    carry = 0\n\n    while l1 or l2 or carry:\n        sum_val = (l1.val if l1 else 0) + (l2.val if l2 else 0) + carry\n        carry = sum_val // 10\n        current.next = ListNode(sum_val % 10)\n        current = current.next\n\n        if l1: l1 = l1.next\n        if l2: l2 = l2.next\n\n    return dummy.next\n```\n\n'

In [18]:
import libcst as cst
import re

In [12]:
split_doc = doc.split('```python')

NameError: name 'cst' is not defined

In [29]:
code_list = []
problem_list = []
data_list = []
cst_list = []
bad_code = []
explanation_list = []
for code_with_data,code_with_problem,code,explanation in zip(docs["code_with_data"],docs["code_with_problem"],docs["code_only"],docs["explanation_only"]):
    #remove ```python and ``` at the end 
    good_code= False
    clean_code = re.sub(r'```python', '', code)
    clean_code = re.sub(r'```.*', '', clean_code).strip()

    split_data = code_with_data.split('```python')
    split_problem = code_with_problem.split('```python')
    if len(split_data) > 1 and len(split_problem) > 1:
        data = split_data[0]
        problem = split_problem[0]
        
        try:
            cst_tree = cst.parse_module(clean_code)
            good_code = True
            code_list.append(clean_code)
            cst_list.append(cst_tree)
            data_list.append(data)
            problem_list.append(problem)
            explanation_list.append(explanation)
        except:
            bad_code.append(clean_code)

In [30]:
print(len(code_list),len(cst_list),len(data_list),len(problem_list),len(bad_code),len(explanation_list))

2348 2348 2348 2348 2 2348


In [35]:
#import all bugs from openbugger
from openbugger.bugs.controlflow import ForgettingToUpdateVariableTransformer, InfiniteWhileTransformer, gen_OffByKIndexTransformer
from openbugger.bugs.data import IncorrectVariableInitializationTransformer, VariableNameTypoTransformer, MutableDefaultArgumentTransformer, UseBeforeDefinitionTransformer
from openbugger.bugs.logical import gen_ComparisonTargetTransfomer, ComparisonSwapTransformer
from openbugger.bugs.type import IncorrectTypeTransformer, NonExistingMethodTransformer, SwapForTransformer
from openbugger.bugs.numpy import NumpyArrayCreationTransformer, NumpyMethodMisuseTransformer, NumpyReshapeMisuseTransformer, NumpyArangeMisuseTransformer, NumpyAxisMisuseTransformer

In [36]:
#create a dictionary where keys are bugs type and values are the bug classes
bug_dict = {"controlflow":[ForgettingToUpdateVariableTransformer, InfiniteWhileTransformer, gen_OffByKIndexTransformer],
            "data":[IncorrectVariableInitializationTransformer, VariableNameTypoTransformer, MutableDefaultArgumentTransformer, UseBeforeDefinitionTransformer],
            "logical":[gen_ComparisonTargetTransfomer, ComparisonSwapTransformer],
            "type":[IncorrectTypeTransformer, NonExistingMethodTransformer, SwapForTransformer],
            "numpy":[NumpyArrayCreationTransformer, NumpyMethodMisuseTransformer, NumpyReshapeMisuseTransformer, NumpyArangeMisuseTransformer, NumpyAxisMisuseTransformer]}


In [37]:
from openbugger.bugger import Bugger